In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from autodm.llm import get_llm
from autodm.roll import Dice, DiceType
from autodm.character import Character
from autodm.locaitons import Location, LocationGraph, setup_new_locations
from autodm.adventurelog import AdventureLog
from autodm.storyline import StoryLineWithStoryItems

from rich import print

/Users/michaelfrantz/miniconda3/envs/autodm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
locations = setup_new_locations()

In [4]:
region = locations[locations.list_locations('region').pop()]
city = locations[locations.list_locations('city').pop()]

In [5]:
# building = city.generate_child(type='building', storyline="The adventurers enter the wizard's guild")
# room1 = building.generate_child(storyline="The adventurers find and enter the wizard's library.")
# room2 = building.generate_child(storyline="The adventurers find and enter thewizard's alchemy lab")
# room3 = building.generate_child(storyline="The adventurers find and enter thewizard's training range")
# locations.add_location(building)
# locations.add_location(room1)
# locations.add_location(room2)
# locations.add_location(room3)
# locations.set_current_location(room1.name)

In [6]:
character = Character.generate(level=0)
print(character)

Character(
    chr_class='wizard',
    chr_race='human',
    name='Aldric',
    attributes=Attributes(strength=16, dexterity=10, constitution=12, intelligence=3, wisdom=13, charisma=2),
    level=0,
    max_hp=22,
    hp=22,
    spells=[],
    equipment=[]
)

In [7]:
story = StoryLineWithStoryItems.generate(character, city, region, locations)